In [ ]:
# Download the scPDB dataset and extract the dataset in "data/scPDB/raw"
!aria2c -c -x 8 -s 8 -d "../data/scPDB" http://bioinfo-pharma.u-strasbg.fr/scPDB/ressources/2016/scPDB.tar.gz --out 'scPDB.tar.gz'
!tar xvzf ../data/scPDB/scPDB.tar.gz -C ../data/scPDB/raw/

In [ ]:
# For 10-fold Cross Validation, we will use the splits that were generated by https://arxiv.org/abs/1904.06517
!aria2c -c -x 8 -s 8 -d "../data/scPDB" https://gitlab.com/cheminfIBB/kalasanty/-/archive/master/kalasanty-master.tar.gz?path=data --out 'kalasanty-master-data.tar.gz'
!tar xvzf ../data/scPDB/kalasanty-master-data.tar.gz -C ../data/scPDB/

In [ ]:
# Some constants that will be required
# ALWAYS RUN THIS CODE CELL
import os
from glob import glob

data_dir = os.path.abspath("../data/scPDB")
raw_dir = os.path.join(data_dir, "raw")
pssm_dir = os.path.join(data_dir, "pssm")
splits_dir = os.path.join(data_dir, "splits")
preprocessed_dir = os.path.join(data_dir, "preprocessed")

In [ ]:
# For sequence-based prediction, we need to use RCSB FASTA files and since scPDB has only mol2 files for the proteins, we will download the fasta file and PDB files of the given proteins ourself. This allows for proper calculation of the labels
# Note that the downloaded PDB automatically has all the structures of a particular PDB ID
# Hence, we just use the first structure instead of all
# Download sequence and PDB files from RCSB for easier matching of labels
import urllib

for file in sorted(os.listdir(raw_dir)):
    pdb_id = file[:4]
    print(pdb_id)

    pdb_save = path.join(folder, file, "downloaded.pdb")
    if not path.exists(pdb_save):
        try:
            urllib.request.urlretrieve('http://files.rcsb.org/download/' + pdb_id + ".pdb", pdb_save)
        except:
            print("Err: pdb " + pdb_id)

    fasta_save = path.join(folder, file, "sequence.fasta")
    if not path.exists(fasta_save):
        try:
            urllib.request.urlretrieve('https://www.rcsb.org/pdb/download/downloadFastaFiles.do?structureIdList=' + pdb_id + '&compressionType=uncompressed', fasta_save)
        except:
            print("Err: fasta " + pdb_id)

In [ ]:
# Check whether downloaded PDB and sequence files are correct
# A few of the PDB files have been obseleted and hence will have to manually download them
import subprocess

def last_line(f):
    proc = subprocess.Popen(['tail', '-n', "1", f], stdout=subprocess.PIPE)
    line = proc.stdout.readlines()[0].decode("utf-8").strip()
    return line

for file in sorted(os.listdir(raw_dir)):
    pdb_id = file[:4]
    pdb_save = os.path.join(raw_dir, file, "downloaded.pdb")
    if last_line(pdb_save) != "END":
        print("Err: PDB " + file)
    fasta_save = os.path.join(raw_dir, file, "sequence.fasta")
    with open(fasta_save, "r") as f:
        line = f.readline()
        if line[0] != ">":
            print("Err: FASTA " + file)

In [ ]:
# We need to generate MSAs for the protein sequences in the dataset
# For that, we need to split the sequence.fasta file into respective chain.fasta files
# Also, we need to remove the fasta files of DNA/RNA seqeuences

for file in sorted(os.listdir(raw_dir)):
    file = file.strip()
    pre = os.path.join(raw_dir, file)
    
    # Read SEQRES entries in PDB file to determine whether a chain
    # has a protein sequence or not
    pdb_file = os.path.join(pre, "downloaded.pdb")
    do_not_include = set()
    with open(pdb_file, "r") as f:
        line = f.readline()
        while line[:6] != "SEQRES":
            line = f.readline()
        while line[:6] == "SEQRES":
            chain_id = line[11]
            residue = line[19:22]
            # Generally DNA/RNA have 1 or 2-letter codes
            if " " in residue:
                do_not_include.add(chain_id)
            line = f.readline()
    
    fasta = os.path.join(pre, "sequence.fasta")
    with open(fasta, "r") as f:
        header = f.readline()
        while 1:
            chain_id = header[6:7]
            sequence = ""
            line = f.readline()
            while line != "" and line is not None and line[0] != ">":
                sequence += line.strip()
                line = f.readline()
            if chain_id not in do_not_include:
                with open(os.path.join(pre, chain_id + ".fasta"), "w") as hlp:
                    hlp.write(header)
                    hlp.write(sequence + "\n")
            if line == "" or line is None:
                break
            header = line

In [ ]:
# In case you want to delete the generated fasta files from the above cell, use this
# for file in sorted(os.listdir(raw_dir)):
#     for fasta in glob(os.path.join(raw_dir, file.strip(), "?.fasta")):
#         os.remove(fasta)

In [ ]:
# The fasta files generated will have a lot of common sequences
# To speed up MSA generation, let us create a unique file that has common sequences
# Then we can generate the MSAs for only the first chain in every line
from collections import defaultdict

sequences = defaultdict(list)
for file in sorted(os.listdir(raw_dir)):
    pre = os.path.join(raw_dir, file.strip())
    for fasta in sorted(os.listdir(pre)):
        if fasta[2:] != "fasta":
            continue
        chain_id = fasta[0]
        with open(os.path.join(pre, fasta)) as f:
            f.readline()
            sequence = f.readline().strip()
            # This choice was made so that rsync would work much better and easier
            sequences[sequence].append(file + "/" + chain_id + "*")

keys = list(sequences.keys())

with open(os.path.join(data_dir, "unique"), "w") as f:
    for key in keys:
        line = ""
        for chain_id in sequences[key]:
            line += chain_id + " "
        f.write(line[:-1] + "\n")

In [ ]:
# Run a regex search on the generated fasta files to ensure that we don't have any DNA/RNA sequences
# Will have to manually check the files to ensure that they are protein sequences
# All of them are protein sequences

import re

def match(strg, search=re.compile(r"[^ACGTURYKMSWBDHVN\-\.]").search):
    return not bool(search(strg))

with open(os.path.join(data_dir, "unique"), "r") as f:
    lines = f.readlines()

for line in lines:
    pdb_id_struct, chain_id = line.strip().split()[0].split("/")
    chain_id = chain_id[0]
    with open(os.path.join(raw_dir, pdb_id_struct, chain_id + ".fasta"), "r") as f:
        f.readline()
        seq = f.readline().strip()
    if match(seq):
        print(pdb_id_struct, chain_id)

In [ ]:
# MSAs need to be generated for the fasta files
# Refer to https://github.com/crvineeth97/msa-generator

In [ ]:
# MAKING OF .npz FILES FROM HERE
# Let us preprocess ALL the available data and create .npz files which contain pdb_id, chain_id, sequence, length, labels
import numpy as np

# Write the code for this here


In [21]:
# Extract the selected features of amino acids
# Code here

def get_amino_acid_features(csv_file):
    feats = {}
    with open(csv_file) as f:
        records = csv.reader(f)
        for i, row in enumerate(records):
            if i == 0:
                length = len(row) - 1
                continue
            feats[row[0]] = np.array(
                [float(el) if el != "" else 0.0 for el in row[1:]], dtype=np.float32
            )
        feats["X"] = np.zeros(length)
    feats = defaultdict(lambda: np.zeros(length), feats)
    return feats



In [32]:
# Assuming that all the PSSMs have been copied to data/scPDB/pssm
# We can have more features included as well. For now, let us consider PSSMs
# !rsync -avP --include="*/" --include="*pssm" --exclude="*" ~/Git/msa-generator/data/scPDB/ ~/Git/protein-binding-site-prediction/data/scPDB/pssm/
# Now, let us preprocess the files again to generate the features directly that can be imported into pytorch easily

# USING CONCATENATION STRATEGY

from collections import defaultdict
import numpy as np
import csv

# List of amino acids and their integer representation
AA_ID_DICT = {
    "X": 0,
    "A": 1,
    "C": 2,
    "D": 3,
    "E": 4,
    "F": 5,
    "G": 6,
    "H": 7,
    "I": 8,
    "K": 9,
    "L": 10,
    "M": 11,
    "N": 12,
    "P": 13,
    "Q": 14,
    "R": 15,
    "S": 16,
    "T": 17,
    "V": 18,
    "W": 19,
    "Y": 20,
}
AA_ID_DICT = defaultdict(lambda: 0, AA_ID_DICT)

# One-hot encoding and positional encoding
feat_vec_len = 21
feat_vec_len += 1

# We generated PSSMs for select sequences
# Mapping different sequences to the one for which we generated
common_pssms = defaultdict(str)
with open(os.path.join(data_dir, "unique"), "r") as f:
    for line in f.readlines():
        line = line.strip().split()
        pssm_generated_for = line[0][:-1]
        common_pssms[pssm_generated_for] = pssm_generated_for
        for pdb_id_struct_chain in line[1:]:
            pdb_id_struct_chain = pdb_id_struct_chain[:-1]
            common_pssms[pdb_id_struct_chain] = pssm_generated_for


def get_pssm(pdb_id_struct, chain_id, length):
    pssm_pdb_id_struct, pssm_chain_id = common_pssms[pdb_id_struct + "/" + chain_id].split("/")
    with open(os.path.join(pssm_dir, pssm_pdb_id_struct, pssm_chain_id + ".pssm"), "r") as f:
        lines = f.readlines()
    feature = np.zeros((21, length))
    for i, line in enumerate(lines):
        feature[i] = np.array(line.strip().split(), dtype=np.float32)
    return feature

# PSSM length
feat_vec_len += 21

# Amino acid physico-chemical features selected by removing highly correlated features
AA_sel_feats = get_amino_acid_features(os.path.join(data_dir, "selected_features.csv"))
feat_vec_len += len(AA_sel_feats["X"])


def generate_input(sample):
    """
    Generate input for a single sample which is a dictionary containing required items
    """
    X = np.zeros((feat_vec_len, sample["length"]))

    # One-hot encoding
    X[:21] = np.array([np.eye(21)[AA_ID_DICT[el]] for el in sample["sequence"]]).T

    # Positional encoding
    X[21] = np.arange(1, sample["length"] + 1, dtype=np.float32) / sample["length"]

    # PSSM
    X[22:43] = sample["pssm"]

    # AA Properties
    X[43:] = np.array([AA_sel_feats[aa] for aa in sample["sequence"]]).T

    return X


for pdb_id_struct in sorted(os.listdir(preprocessed_dir)):
    flg = True
    pre = os.path.join(preprocessed_dir, pdb_id_struct)
    features_file = os.path.join(pre, "features.npy")
    labels_file = os.path.join(pre, "labels.npy")

    if os.path.exists(labels_file):
        continue
    print(pdb_id_struct)

    for file in sorted(os.listdir(pre)):
        # In case features were generated but not labels, redo it
        if file == "features.npz":
            continue
        chain_id = file[-len(".npz") - 1 : -len(".npz")]
        sample = np.load(os.path.join(pre, file))
        sample = {
            key: sample[key].item() if sample[key].shape is () else sample[key]
            for key in sample
        }
        sample["pssm"] = get_pssm(pdb_id_struct, chain_id, sample["length"])
        if flg:
            X = generate_input(sample)
            y = sample["labels"]
            flg = False
        else:
            # Using concatenation strategy
            tmp = generate_input(sample)
            X = np.concatenate((X, tmp), 1)
            y = np.concatenate((y, sample["labels"]), 0)

    np.save(features_file, X)
    np.save(labels_file, y)

# ISSUES

<input type="checkbox"> Improve downloading PDB and fasta by using something that resumes downloads

<input type="checkbox"> Not sure if checking for a space in the residue is the best way of checking. Can use the code_with_modified_residues dictionary from NWalign.py (https://zhanglab.ccmb.med.umich.edu/NW-align/NWalign.py)

<input type="checkbox"> Make the pdb_id field in the preprocessed files into pdb_id_struct

